In [3]:
pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import requests

# 데이터프레임 불러오기
df = pd.read_excel('선박 화재 발생 일자 데이터.xlsx', header=None)
df['year'] = df[0].dt.year
df['month'] = df[0].dt.month
df['day'] = df[0].dt.day
df = df.drop(0, axis=1)

apikey = "ikyOZ5uYRSGMjmebmJUhoQ"
stn_dic = {
    "부산광역시": 159,
    "전라남도": 168,
    "경상남도": 159,
    "강원특별자치도": 119,
    "경상북도": 143,
    "전북특별자치도": 146,
    "인천광역시": 112,
    "제주특별자치도": 184,
    "서울특별시": 108,
    "울산광역시": 152,
    "경기도": 119,
    "충청남도": 177
}

In [2]:
import requests
import time

def download_file(date, stn):
    url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm={date}&stn={stn}&help=0&authKey={apikey}'
    # print(url)
    for attempt in range(5):  # 5회 시도
        try:
            response = requests.get(url, timeout=10)  # 10초 타임아웃
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
            return response.content.decode('utf-8').splitlines()  # 성공적으로 데이터를 가져왔을 때
        except requests.exceptions.Timeout:
            print(f"Attempt {attempt + 1} timed out, retrying...")
            time.sleep(1)  # 1초 대기 후 재시도
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {date}, station {stn}: {e}")
            return None
    
    print(f"Failed to fetch data for {date}, station {stn} after multiple attempts.")
    return None


# 파일에서 읽어오기

In [20]:
def extract_weather_data(lines):
    split_data = lines.split()  # 공백 기준으로 split
    if split_data[0] == '#7777END':  # 데이터가 없는 경우
        return None
    else:
        return {
            'ws': split_data[3],  # 풍속(m/s)
            'pa': split_data[7], # 기압(hPa)
            'ps': split_data[8], # 해면 기압(hPa)
            'ta': split_data[11], # 기온(C)
            'td': split_data[12], # 이슬점온도(C)
            'hm': split_data[13], # 상대습도(%)
            'rn': split_data[16],  # 일일 강수량(mm)
            'ss': split_data[33], # 일조 (hr)
            'si': split_data[34], # 일사 (MJ/m2)
            'ts': split_data[36] # 지면온도 (C)
        }
    return None

# 파일 경로 지정
file_path = 'output_file충청남도.txt'  # 파일 경로를 지정하세요

# 파일 읽기
with open(file_path, 'r') as file:
    lines = file.readlines()  # 모든 줄을 리스트로 읽어옴

# 5번째 줄 가져오기 (인덱스는 0부터 시작하므로 4로 접근)
fifth_line = lines[4].strip()  # .strip()을 사용하여 줄바꿈 문자 제거

temp = extract_weather_data(fifth_line)

temp


{'ws': '3.9',
 'pa': '1021.5',
 'ps': '1030.2',
 'ta': '4.2',
 'td': '-11.8',
 'hm': '30.0',
 'rn': '-9.0',
 'ss': '1.0',
 'si': '2.12',
 'ts': '17.8'}

# GET요청 후 바로 처리

In [7]:
def extract_weather_data(lines):
    if lines and len(lines) > 4:
        fourth_line = lines[4].strip()  # 5번째 줄 추출
        split_data = fourth_line.split()  # 공백 기준으로 split
        if split_data[0] == '#7777END':  # 데이터가 없는 경우
            return None
        else:
            return {
            'ws': split_data[3],  # 풍속(m/s)
            'pa': split_data[7], # 기압(hPa)
            'ps': split_data[8], # 해면 기압(hPa)
            'ta': split_data[11], # 기온(C)
            'td': split_data[12], # 이슬점온도(C)
            'hm': split_data[13], # 상대습도(%)
            'rn': split_data[16],  # 일일 강수량(mm)
            'ss': split_data[33], # 일조 (hr)
            'si': split_data[34], # 일사 (MJ/m2)
            'ts': split_data[36] # 지면온도 (C)
            }
    return None

In [4]:
from tqdm import tqdm

# 날씨 정보를 데이터프레임에 추가하는 함수
def add_weather_data_to_df(df):
    weather_data = []
    for i in tqdm(range(len(df)), desc="Processing rows", unit="row"):
        region = df.iloc[i][1]
        date = f"{df.iloc[i]['year']}{str(df.iloc[i]['month']).zfill(2)}{str(df.iloc[i]['day']).zfill(2)}1300"
        
        # 지역에 맞는 날씨 정보 요청
        lines = download_file(date, str(stn_dic.get(region, '108')))
        
        # 날씨 데이터 추출
        weather = extract_weather_data(lines)
        
        # 데이터가 없으면 서울 지역으로 요청
        if weather is None:
            lines = download_file(date, '108')
            weather = extract_weather_data(lines)
        
        # 결과 추가 (날씨 데이터를 리스트에 추가)
        if weather:
            weather_data.append(weather)
        else:
            weather_data.append({'ws': None, 'ta': None, 'hm': None, 'rn': None})
    
    # 각 열을 추가
    df['ws'] = [w['ws'] for w in weather_data]
    df['ta'] = [w['ta'] for w in weather_data]
    df['hm'] = [w['hm'] for w in weather_data]
    df['rn'] = [w['rn'] for w in weather_data]
    
    return df

In [7]:
# 날씨 데이터를 데이터프레임에 추가
df_with_weather = add_weather_data_to_df(df)

# 결과 확인
display(df_with_weather)

Processing rows: 100%|██████████| 596/596 [03:38<00:00,  2.73row/s]


,1,2,year,month,day,ws,ta,hm,rn
0,부산광역시,영도구,2019,1,2,3.9,4.2,30.0,-9.0
1,부산광역시,영도구,2019,1,3,5.1,5.3,20.0,-9.0
2,전라남도,고흥군,2019,1,3,1.4,4.6,45.0,-9.0
3,전라남도,목포시,2019,1,5,4.4,7.0,40.0,-9.0
4,경상남도,거제시,2019,1,10,4.0,7.5,21.0,-9.0
...,...,...,...,...,...,...,...,...,...
591,경상남도,창원시 마산합포구,2023,11,24,4.3,8.5,20.0,-9.0
592,제주특별자치도,제주시,2023,12,7,3.6,16.9,33.0,-9.0
593,전라남도,목포시,2023,12,16,10.6,3.4,60.0,0.2
594,전라남도,여수시,2023,12,19,1.9,4.8,44.0,0.3


In [8]:
df_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       596 non-null    object
 1   2       596 non-null    object
 2   year    596 non-null    int32 
 3   month   596 non-null    int32 
 4   day     596 non-null    int32 
 5   ws      596 non-null    object
 6   ta      596 non-null    object
 7   hm      596 non-null    object
 8   rn      596 non-null    object
dtypes: int32(3), object(6)
memory usage: 35.0+ KB


In [9]:
df_with_weather.head()

,1,2,year,month,day,ws,ta,hm,rn
0,부산광역시,영도구,2019,1,2,3.9,4.2,30.0,-9.0
1,부산광역시,영도구,2019,1,3,5.1,5.3,20.0,-9.0
2,전라남도,고흥군,2019,1,3,1.4,4.6,45.0,-9.0
3,전라남도,목포시,2019,1,5,4.4,7.0,40.0,-9.0
4,경상남도,거제시,2019,1,10,4.0,7.5,21.0,-9.0


In [10]:
# -9.0 값을 0으로 변경
df_with_weather['rn'] = df_with_weather['rn'].replace('-9.0', '0')
# 컬럼을 int로 변환
df_with_weather['rn'] = df_with_weather['rn'].astype(float)

In [11]:
df_with_weather.head()

,1,2,year,month,day,ws,ta,hm,rn
0,부산광역시,영도구,2019,1,2,3.9,4.2,30.0,0.0
1,부산광역시,영도구,2019,1,3,5.1,5.3,20.0,0.0
2,전라남도,고흥군,2019,1,3,1.4,4.6,45.0,0.0
3,전라남도,목포시,2019,1,5,4.4,7.0,40.0,0.0
4,경상남도,거제시,2019,1,10,4.0,7.5,21.0,0.0


In [12]:
df_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       596 non-null    object 
 1   2       596 non-null    object 
 2   year    596 non-null    int32  
 3   month   596 non-null    int32  
 4   day     596 non-null    int32  
 5   ws      596 non-null    object 
 6   ta      596 non-null    object 
 7   hm      596 non-null    object 
 8   rn      596 non-null    float64
dtypes: float64(1), int32(3), object(5)
memory usage: 35.0+ KB


In [13]:
df_with_weather = df_with_weather.drop(1, axis=1).drop(2, axis=1)
# 결과 컬럼 추가 및 화재 발생(1)으로 기록
df_with_weather['result'] = 1
df_with_weather.head()

,year,month,day,ws,ta,hm,rn,result
0,2019,1,2,3.9,4.2,30.0,0.0,1
1,2019,1,3,5.1,5.3,20.0,0.0,1
2,2019,1,3,1.4,4.6,45.0,0.0,1
3,2019,1,5,4.4,7.0,40.0,0.0,1
4,2019,1,10,4.0,7.5,21.0,0.0,1


In [17]:
df_with_weather.to_excel('선박 화재 발생시 날씨 정보.xlsx', sheet_name='WeatherData', index=False)

# 화재 x 날씨 데이터

In [71]:
import pandas as pd
from tqdm import tqdm 

# 빈 데이터프레임 초기화
df = pd.DataFrame(columns=['year', 'month', 'day', 'ws', 'ta', 'hm', 'rn', 'result'])

# 연도, 월, 일 범위 설정
years = range(2019, 2024)
months = range(1, 13)
days = range(1, 32)

# tqdm으로 진행 바 추가
total_days = len(years) * len(months) * len(days)
with tqdm(total=total_days, desc="Processing dates", unit="day") as pbar:
    i = 0
    for y in years:
        for m in months:
            for d in days:
                # 날짜 형식을 yymmdd로 변환
                target_date = f"{str(y)}{str(m).zfill(2)}{str(d).zfill(2)}1300"    
                com_date = f"{str(df_with_weather.iloc[i]['year'])}{str(df_with_weather.iloc[i]['month']).zfill(2)}{str(df_with_weather.iloc[i]['day']).zfill(2)}"
                
                # 만약 화재가 발생한 날짜라면 건너뛴다.
                if target_date == com_date:
                    i += 1
                    continue
                else:
                    # 날씨 데이터 다운로드
                    lines = download_file(target_date, str(159))
                    # display(lines)
                    # 날씨 데이터 추출
                    weather = extract_weather_data(lines)
                    # print(target_date)
                    # print(weather['ws'] + ' ' + weather['ta'] + ' ' + weather['hm'] + ' ' + weather['rn'])

                    # 새로운 데이터를 리스트에 추가
                    if weather:
                        new_data = {
                            'year': y, 'month': m, 'day': d, 
                            'ws': weather['ws'], 'ta': weather['ta'], 
                            'hm': weather['hm'], 'rn': weather['rn'], 
                            'result': 0
                        }
                    else:
                        new_data = {
                            'year': y, 'month': m, 'day': d, 
                            'ws': None, 'ta': None, 
                            'hm': None, 'rn': None, 
                            'result': 0
                        }
                    
                    # 데이터프레임에 새 데이터 추가
                    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)

                # 진행 바 업데이트
                pbar.update(1)
# 결과 데이터프레임 출력
print(df.head())


Processing dates: 100%|██████████| 1860/1860 [12:30<00:00,  2.48day/s]

   year month day   ws   ta    hm    rn result
0  2019     1   1  3.0  4.1  36.0  -9.0      0
1  2019     1   2  3.9  4.2  30.0  -9.0      0
2  2019     1   3  5.1  5.3  20.0  -9.0      0
3  2019     1   4  5.8  6.3  20.0  -9.0      0
4  2019     1   5  2.5  8.5  30.0  -9.0      0


### 2월 30일 같은 없는 날짜의 값을 제거한다.

In [74]:
df = df.dropna()

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1825 entries, 0 to 1859
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    1825 non-null   object
 1   month   1825 non-null   object
 2   day     1825 non-null   object
 3   ws      1825 non-null   object
 4   ta      1825 non-null   object
 5   hm      1825 non-null   object
 6   rn      1825 non-null   object
 7   result  1825 non-null   object
dtypes: object(8)
memory usage: 128.3+ KB


In [95]:
# -9.0 값을 0으로 변경
df['rn'] = df['rn'].replace('-9.0', '0')
# 컬럼을 int로 변환
df['rn'] = df['rn'].astype(float)

In [101]:
df.sample(10)

,year,month,day,ws,ta,hm,rn,result
420,2020,2,18,3.3,4.0,30.0,0.0,0
646,2020,9,27,3.1,24.6,54.0,0.0,0
1106,2021,12,22,1.8,13.5,27.0,0.0,0
1704,2023,7,31,4.1,31.7,60.0,0.0,0
271,2019,9,24,3.5,23.5,58.0,0.0,0
648,2020,9,29,2.7,25.3,44.0,0.0,0
1734,2023,8,30,3.5,28.1,78.0,6.6,0
42,2019,2,12,2.0,7.9,29.0,0.0,0
1747,2023,9,12,2.6,29.1,67.0,0.0,0
230,2019,8,14,5.4,32.6,63.0,0.0,0


In [102]:
df.to_excel('선박 화재 발생x 날씨 정보.xlsx', sheet_name='WeatherData', index=False)

# 앞으로 할 일 
화재 발생시 일자가 겹치는 컬럼이 다수 존재 -> 중복 제거    
화재 발생x 데이터에서는 2019~2024모든 일의 컬럼이 존재 -> 화재 발생시 데이터를 삽입하며 화재 발생 x에 존재하던 해당 일자의 데이터를 삭제 후 삽입한다.

In [1]:
import pandas as pd

In [3]:
df = pd.read_excel('선박 화재 발생시 날씨 정보.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    596 non-null    int64  
 1   month   596 non-null    int64  
 2   day     596 non-null    int64  
 3   ws      596 non-null    float64
 4   ta      596 non-null    float64
 5   hm      596 non-null    float64
 6   rn      596 non-null    float64
 7   result  596 non-null    int64  
dtypes: float64(4), int64(4)
memory usage: 37.4 KB


In [17]:
# 중복 확인을 위한 기준이 year, month, day인 경우
def select_row_based_on_conditions(group):
    if group.empty:
        return None  # 빈 그룹 처리

    # 조건 1: hm가 낮고 rn가 낮으며 ta가 높은 행 선택
    selected_rows = group.loc[(group['hm'] == group['hm'].min()) &
                              (group['rn'] == group['rn'].min()) &
                              (group['ta'] == group['ta'].max())]
    
    # 조건 2: 만약 선택된 행들이 여러 개라면, ws가 높은 행 선택
    if len(selected_rows) > 1:
        selected_rows = selected_rows.loc[selected_rows['ws'] == selected_rows['ws'].max()]
    
    # 조건 3: 그래도 여러 행이 남아있다면, 첫 번째 행을 선택
    return selected_rows.iloc[0] if not selected_rows.empty else group.iloc[0]

# year, month, day로 그룹핑 후 조건에 맞는 행만 남김
df_cleaned = df.groupby(['year', 'month', 'day'], group_keys=False).apply(select_row_based_on_conditions)

# 결측값(빈 그룹에서 반환된 None) 제거
df_cleaned = df_cleaned.dropna()

# 결과 확인
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 509 entries, (2019, 1, 2) to (2023, 12, 21)
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    509 non-null    float64
 1   month   509 non-null    float64
 2   day     509 non-null    float64
 3   ws      509 non-null    float64
 4   ta      509 non-null    float64
 5   hm      509 non-null    float64
 6   rn      509 non-null    float64
 7   result  509 non-null    float64
dtypes: float64(8)
memory usage: 33.8 KB


In [55]:
# 중복 처리된 데이터프레임을 저장
df_cleaned.to_excel('중복_제거_선박_화재_날씨_정보.xlsx', index=False)

# 데이터 프레임 합치기

In [56]:
import pandas as pd

In [63]:
df_o = pd.read_excel('중복_제거_선박_화재_날씨_정보.xlsx')
df_x = pd.read_excel('선박 화재 발생x 날씨 정보.xlsx')

In [64]:
display(len(df_o))
display(len(df_x))

509

1825

In [66]:
# 두 DataFrame을 세로로 병합합니다.
df_merged = pd.concat([df_x, df_o], ignore_index=True)
display(df_merged.info())
df_merged.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    2334 non-null   int64  
 1   month   2334 non-null   int64  
 2   day     2334 non-null   int64  
 3   ws      2334 non-null   float64
 4   ta      2334 non-null   float64
 5   hm      2334 non-null   float64
 6   rn      2334 non-null   float64
 7   result  2334 non-null   int64  
dtypes: float64(4), int64(4)
memory usage: 146.0 KB


None

,year,month,day,ws,ta,hm,rn,result
0,2019,1,1,3.0,4.1,36.0,0.0,0
1,2019,1,2,3.9,4.2,30.0,0.0,0
2,2019,1,3,5.1,5.3,20.0,0.0,0
3,2019,1,4,5.8,6.3,20.0,0.0,0
4,2019,1,5,2.5,8.5,30.0,0.0,0


In [68]:
# year, month, day가 중복되는 행 중 result가 0인 행 삭제
df_merged = df_merged[~((df_merged.duplicated(subset=['year', 'month', 'day'], keep=False)) & (df_merged['result'] == 0))]
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1825 entries, 0 to 2333
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1825 non-null   int64  
 1   month   1825 non-null   int64  
 2   day     1825 non-null   int64  
 3   ws      1825 non-null   float64
 4   ta      1825 non-null   float64
 5   hm      1825 non-null   float64
 6   rn      1825 non-null   float64
 7   result  1825 non-null   int64  
dtypes: float64(4), int64(4)
memory usage: 128.3 KB


In [69]:
# year, month, day 기준으로 정렬
df_sorted = df_merged.sort_values(by=['year', 'month', 'day'])
df_sorted.head()

,year,month,day,ws,ta,hm,rn,result
0,2019,1,1,3.0,4.1,36.0,0.0,0
1825,2019,1,2,3.9,4.2,30.0,0.0,1
1826,2019,1,3,5.1,5.3,20.0,0.0,1
3,2019,1,4,5.8,6.3,20.0,0.0,0
1827,2019,1,5,4.4,7.0,40.0,0.0,1


In [70]:
# 데이터프레임을 저장
df_sorted.to_excel('최종본.xlsx', index=False)

# 이상치 제거

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('최종본.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1825 non-null   int64  
 1   month   1825 non-null   int64  
 2   day     1825 non-null   int64  
 3   ws      1825 non-null   float64
 4   ta      1825 non-null   float64
 5   hm      1825 non-null   int64  
 6   rn      1825 non-null   float64
 7   result  1825 non-null   int64  
dtypes: float64(3), int64(5)
memory usage: 114.2 KB


In [9]:
# -9.0을 0으로 바꾸기
df['ws'] = df['ws'].replace(-9.0, 0)
df['hm'] = df['hm'].replace(-9.0, 0)

In [10]:
# 데이터프레임을 저장
df.to_excel('최종본.xlsx', index=False)